In [1]:
# Point Python to the path where we have installed the bad and darc packages
import sys
sys.path.insert(0, '/Users/btvincent/git-local/darc-experiments-python')

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm

import darc
from darc.delayed import models
from darc.designs import BADDesignGenerator

from darc.data_plotting import plot_delay_without_front_end_delays
from darc_parameter_recovery import simulated_experiment_trial_loop

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

## Simulated experiment with `DelaySlice` model

In [3]:
max_trials = 20

design_thing = BADDesignGenerator(max_trials=max_trials,
                          RA=list(np.arange(0,100,1)),
                          DB=[180])

model = models.DelaySlice(n_particles=50_000)

# model = model.generate_faux_true_params()
indiff_true = 0.8
α_true = 0.05
model.θ_true = pd.DataFrame.from_dict({'indiff': [indiff_true], 'α': [α_true]})

fitted_model = simulated_experiment_trial_loop(design_thing, model)


KeyError: 'logk'

Visualise the raw data

In [ ]:
fig, ax = plt.subplots(figsize=(9,4))
plot_delay_without_front_end_delays(ax, design_thing.get_df())
ax.set_xlim([0, 365])
ax.set_ylim([0, 1])

## Posterior predictive checking

In [ ]:
def plot_data(ax, all_data):
    x = all_data['RA'].values / all_data['RB'].values
    y = all_data['R'].values
    ax.scatter(x, y, c='k')

In [ ]:
Phi = lambda x: norm.cdf(x, loc=0, scale=1)

def Psi(x, indiff, α, ϵ=0.01):
    return ϵ+(1-2*ϵ) * Phi((indiff-x)/α) # TODO: CHECK x is right way around

In [ ]:
θpe = fitted_model.get_θ_point_estimate()
indiff_median = θpe['indiff'].values
α_median = θpe['α'].values

In [ ]:
x = np.linspace(0, 1, 500)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,12))

# Psychometric function posterior predictive check + data ====================
XN = 500
x = np.linspace(0, 1, XN)

# plot a bunch of samples from the posterior distribution
N_SAMPLES = 300
θ = fitted_model.θ.sample(N_SAMPLES)
y = np.zeros((XN, N_SAMPLES))
for i, (indiff, α) in enumerate(zip(θ['indiff'].values, θ['α'].values)):
    y[:,i] = Psi(x, indiff, α, ϵ=0.01)
    
ax[0].plot(x, y, c='k', alpha=0.1)

# plot true psychophysical function
y = Psi(x, indiff_true, α_true, ϵ=0.01)
ax[0].plot(x, y, lw=4, label='true')

# plot posterior median
y = Psi(x, indiff_median, α_median, ϵ=0.01)
ax[0].plot(x, y, lw=4, label='posterior median')

# overlay the data onto this plot
plot_data(ax[0], design_thing.get_df())

# formatting
ax[0].set_ylim([0-0.05, 1+0.05])
ax[0].legend()

ax[0].set_xlabel('indifference point = $R^A/R^B$')
ax[0].set_ylabel('P(choose B)')
ax[0].set_title('Posterior predictive check')


# Visualise things as a function of trial ===================================

trial = np.arange(design_thing.get_df()shape[0]) + 1
indiff = design_thing.get_df()['RA'].values / design_thing.get_df()['RB'].values
ax[1].plot(indiff, trial, '-o', ms=7, c='k', label='data')

ax[1].set_ylabel('trial')
ax[1].set_xlabel('$R^A/R^B$')
ax[1].set_yticks(trial)
ax[1].legend()
ax[1].set_xlim([0-0.05, 1+0.05])
ax[1].set_title('Experimental designs')

plt.subplots_adjust(hspace=0.25)

In [ ]:
from scipy.stats import halfnorm
α = halfnorm(loc=0, scale=0.05)
plt.hist(α.rvs(10000), 50, density=True, facecolor='g', alpha=0.75);